In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, accuracy_score, f1_score
import numpy as np


2023-11-12 15:25:13.268498: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-12 15:25:13.934938: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/roronoa/.conda/envs/tf/lib/python3.7/site-packages/nvidia/cudnn/lib:/home/roronoa/.conda/envs/tf/lib/:
2023-11-12 15:25:13.935014: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/roronoa/.conda/

In [2]:
# Define the image dimensions and batch size
img_size = (224, 224)
batch_size = 128

In [3]:
# Create data generators for training and testing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
train_data = train_datagen.flow_from_directory(
    '../Data/images/train',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_data = test_datagen.flow_from_directory(
    '../Data/images/test',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 21077 images belonging to 4 classes.
Found 5140 images belonging to 4 classes.


In [5]:
# Create the InceptionV3 base model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)  # Additional dense layer
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
predictions = Dense(4, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

2023-11-12 15:25:15.438424: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-12 15:25:15.461993: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-12 15:25:15.462281: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-12 15:25:15.462893: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [6]:
# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
# Train the model
epochs = 50
history = model.fit(train_data, epochs=epochs)


Epoch 1/50


2023-11-12 15:25:21.941713: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8500
2023-11-12 15:25:22.230273: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-12 15:25:22.846869: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 666.04MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-12 15:25:22.846959: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 666.04MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-11-12 15:25:22.904802: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allo

165/165 [==============================] - 141s 802ms/step - loss: 1.7475 - accuracy: 0.3075
Epoch 2/50
165/165 [==============================] - 128s 772ms/step - loss: 1.4091 - accuracy: 0.3664
Epoch 3/50
165/165 [==============================] - 127s 769ms/step - loss: 1.3153 - accuracy: 0.3962
Epoch 4/50
165/165 [==============================] - 127s 767ms/step - loss: 1.2821 - accuracy: 0.4117
Epoch 5/50
165/165 [==============================] - 127s 770ms/step - loss: 1.2614 - accuracy: 0.4282
Epoch 6/50
165/165 [==============================] - 127s 767ms/step - loss: 1.2513 - accuracy: 0.4322
Epoch 7/50
165/165 [==============================] - 128s 771ms/step - loss: 1.2348 - accuracy: 0.4465
Epoch 8/50
165/165 [==============================] - 129s 782ms/step - loss: 1.2251 - accuracy: 0.4529
Epoch 9/50
165/165 [==============================] - 128s 772ms/step - loss: 1.2223 - accuracy: 0.4550
Epoch 10/50
165/165 [==============================] - 127s 766ms/step - lo

In [8]:
# Predict class labels for the test data
y_pred = model.predict(test_data)
y_true = test_data.labels

# Calculate F1 score and classification report
f1 = f1_score(y_true, y_pred.argmax(axis=1), average='weighted')
accuracy = accuracy_score(y_true, y_pred.argmax(axis=1))
report = classification_report(y_true, y_pred.argmax(axis=1), target_names=train_data.class_indices.keys())

print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)


41/41 [==============================] - 17s 409ms/step
F1 Score: 0.5574
Accuracy: 0.5623
Classification Report:
               precision    recall  f1-score   support

       angry       0.47      0.33      0.39       960
       happy       0.70      0.72      0.71      1825
     neutral       0.50      0.54      0.52      1216
         sad       0.47      0.52      0.49      1139

    accuracy                           0.56      5140
   macro avg       0.54      0.53      0.53      5140
weighted avg       0.56      0.56      0.56      5140

